# Mapa Dinpamico RM

Carlos Olivares <br>
https://github.com/Carlos-Olivares/

In [ ]:
#Importar librerías
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json

%matplotlib inline

In [ ]:
#Importar poligonos de comunas
shapefile = 'ComunasSHP//comunas.shp'

#Leer shapefile con geopandas
gdf = gpd.read_file(shapefile, encoding='utf-8')[['codregion', 'Region', 'cod_comuna', 'Comuna', 'Provincia', 'geometry']]

#Filtramos solo RM
gdf = gdf[gdf['codregion'] == 13]

#Filtramos algunas comunas para visualizar mejor
excluir = [13202, 13203, 13301, 13302, 13303, 13402, 13403, 13404, 13501, 13502, 13503, 13504, 13601, 13602, 13603, 13605]
gdf = gdf[~gdf['cod_comuna'].isin(excluir)]

gdf.head()

In [ ]:
#Columnas de interés
gdf = gdf[['cod_comuna', 'Comuna', 'geometry']]
gdf.head()

In [ ]:
#Habitantes por Comuna
url = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto82/ISCI_weeks.csv'
movilidad = pd.read_csv(url)
movilidad = movilidad[movilidad['region'] == 13].copy()
movilidad.head()

In [ ]:
#Arreglar semana
movilidad['semana'] = (movilidad['semana']+100).astype(str).str[1:]
movilidad.head()

In [ ]:
#Datos Faltantes
movilidad.isna().sum()

In [ ]:
movilidad.dropna(inplace = True)
movilidad.isna().sum()

In [ ]:
#Unir DataFrames
df_final = gdf.merge(movilidad, how = 'inner', left_on = 'cod_comuna', right_on = 'comuna', suffixes = ("","_2"))
display(df_final.head())
print(df_final.shape)

In [ ]:
#Columnas de interés
df_final = df_final[['Comuna', 'semana', 'fecha_inicio', 'fecha_termino', 'var_salidas', 'geometry']]
df_final.head()

In [ ]:
df_final.plot()

# Crear Video Sin Interpolación

In [ ]:
#Crear imágenes
from bokeh.io import output_notebook, show, output_file, save, export_png
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, ColorBar, LinearColorMapper, Label
from bokeh.palettes import cividis
import plotly as plt

#Min-Max Color
min_color = df_final['var_salidas'].min()
max_color = df_final['var_salidas'].max()

#Nombre Mes
meses = {'01': 'Enero',
      '02': 'Febrero',
      '03': 'Marzo',
      '04': 'Abril',
      '05': 'Mayo',
      '06': 'Junio',
      '07': 'Julio',
      '08': 'Agosto',
      '09': 'Septiembre',
      '10': 'Octubre',
      '11': 'Noviembre',
      '12': 'Diciembre'}

#Mapa por semana
for semana in df_final['semana'].unique():
    #Filtrar semana
    df_semana = df_final.query('semana == @semana').copy()
    
    #A json
    merged_json = json.loads(df_semana.to_json())
    
    #Json a String
    json_data = json.dumps(merged_json)
    
    #Cargar la data json_data
    gsource = GeoJSONDataSource(geojson = json_data)

    #Definir una paleta de colores
    colores = cividis(n = 256)

    #Inicializar LinearColorMapper. Para que asocie un número a los colores.
    color_mapper = LinearColorMapper(palette = colores, low = min_color, high = max_color)
    
    #Crear el objeto figura
    fig = figure(title = 'Índice de movilidad por Comuna de Santiago',
              plot_height = 600,
              plot_width = 950,
              toolbar_location = None)
    
    #Formato de Título
    fig.title.text_font_size = '20pt'
    fig.title.align = 'center'

    #Ocultamos los ejes
    fig.axis.visible = False
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    
    #Agregar comunas
    fig.patches('xs', 'ys', 
              source = gsource,
              fill_color = {'field': 'var_salidas', 'transform': color_mapper},
              line_color = 'black',
              line_width = 0.5,
              fill_alpha = 1)
    
    #Anotaciones
    anio = Label(x = -7902000, y=-3919000, text = df_semana['fecha_inicio'].unique()[0][:4], text_font_size='80px')
    fig.add_layout(anio)
    
    mes = Label(x = -7901500, y=-3924000, text = meses[df_semana['fecha_inicio'].unique()[0][5:7]], text_font_size='30px')
    fig.add_layout(mes)
    
    #Rescatar Fechas
    inicio = df_semana['fecha_inicio'].unique()[0]
    termino = df_semana['fecha_termino'].unique()[0]
    fechas = 'Semana de ' + inicio + ' hasta ' + termino
    an_fechas = Label(x = -7849000, y=-3999000, text = fechas, text_font_size='20px')
    fig.add_layout(an_fechas)
    
    #Agregar ColorBar
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))
    fig.add_layout(color_bar, 'right')
    
    #Export as png
    export_png(fig, filename='output\\semana_{}.png'.format(semana))
    print('Semana', semana, 'lista!')

In [ ]:
#Crear Video con OpenCV
#Fuente: https://theailearner.com/2018/10/15/creating-video-from-images-using-opencv-python/

import cv2
import glob
 
img_array = []
for filename in glob.glob('SinInterpolacion/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('final_rapido2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

# Crear Video con Interpolación

In [ ]:
df_final_aux = df_final.copy()
df_final_aux

In [ ]:
#Crear un df con movilidad vacía
n_records = 29
dummy = pd.concat([df_final_aux]*n_records)

#Ordenar por comuna y semana
dummy = dummy.sort_values(['Comuna', 'semana'])

#Reset Index
dummy.reset_index(drop = True, inplace = True)

#Eliminar var_salidas
dummy['var_salidas'] = None

#Agregamos sub_semana
dummy['sub_semana'] = pd.Series(np.tile(np.arange(1, n_records+1), n_records*df_final_aux.shape[0])).astype(int)
dummy.head(60)

In [ ]:
#Setear sub_semana auxiliar
df_final_aux['sub_semana'] = 0

#Merge y ordenar
df_merged = pd.concat([df_final_aux, dummy], axis = 0).sort_values(by=['Comuna', 'semana', 'sub_semana'])
df_merged.reset_index(drop = True, inplace = True)

#Arreglar sub_semana
df_merged['sub_semana'] = (df_merged['sub_semana']+100).astype(str).str[1:]
df_merged.head(60)

In [ ]:
#Interpolar valores faltantes
df_merged['var_salidas'] = df_merged['var_salidas'].astype(float)
df_merged['var_salidas'] = df_merged['var_salidas'].interpolate()

df_merged.head(60)

In [ ]:
df_merged.shape

In [ ]:
df_merged['id_semana'] = list(zip(df_merged['semana'], df_merged['sub_semana']))
df_merged

In [ ]:
from bokeh.io import output_notebook, show, output_file, save, export_png
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, ColorBar, LinearColorMapper, Label
from bokeh.palettes import cividis
import plotly as plt

#Min-Max Color
min_color = df_merged['var_salidas'].min()
max_color = df_merged['var_salidas'].max()

#Nombre Mes
meses = {'01': 'Enero',
      '02': 'Febrero',
      '03': 'Marzo',
      '04': 'Abril',
      '05': 'Mayo',
      '06': 'Junio',
      '07': 'Julio',
      '08': 'Agosto',
      '09': 'Septiembre',
      '10': 'Octubre',
      '11': 'Noviembre',
      '12': 'Diciembre'}

#Mapa por semana
for id_semana in df_merged['id_semana'].unique():
    #Filtrar semana
    df_semana = df_merged.query('id_semana == @id_semana').copy()
    
    #A json
    merged_json = json.loads(df_semana.to_json())
    
    #Json a String
    json_data = json.dumps(merged_json)
    
    #Cargar la data json_data
    gsource = GeoJSONDataSource(geojson = json_data)

    #Definir una paleta de colores
    colores = cividis(n = 256)

    #Inicializar LinearColorMapper. Para que asocie un número a los colores.
    color_mapper = LinearColorMapper(palette = colores, low = min_color, high = max_color)
    
    #Crear el objeto figura
    fig = figure(title = 'Índice de movilidad por Comuna de Santiago',
              plot_height = 600,
              plot_width = 950,
              toolbar_location = None)

    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    fig.title.text_font_size = '20pt'
    fig.title.align = 'center'

    #Ocultamos los ejes
    fig.axis.visible = False

    #Agregar comunas
    fig.patches('xs', 'ys', 
              source = gsource,
              fill_color = {'field': 'var_salidas', 'transform': color_mapper},
              line_color = 'black',
              line_width = 0.5,
              fill_alpha = 1,
              name = 'Comunas')
    
    #Anotaciones
    anio = Label(x = -7902000, y=-3919000, text = df_semana['fecha_inicio'].unique()[0][:4], text_font_size='80px')
    fig.add_layout(anio)
    
    mes = Label(x = -7901500, y=-3924000, text = meses[df_semana['fecha_inicio'].unique()[0][5:7]], text_font_size='30px')
    fig.add_layout(mes)
    
    #Rescatar Fechas
    inicio = df_semana['fecha_inicio'].unique()[0]
    termino = df_semana['fecha_termino'].unique()[0]
    fechas = 'Semana de ' + inicio + ' hasta ' + termino
    an_fechas = Label(x = -7849000, y=-3999000, text = fechas, text_font_size='20px')
    fig.add_layout(an_fechas)
    
    #Agregar ColorBar
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))
    fig.add_layout(color_bar, 'right')

    
    #Display
    export_png(fig, filename='output_interpolado\\semana_{}.png'.format(id_semana))
    print('Semana', id_semana, 'lista!')

In [ ]:
import cv2
import glob
 
img_array = []
for filename in glob.glob('output_interpolado/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('video_interpolado.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()